In [1]:
import numpy as np
import pandas as pd
import warnings
from sklearn.model_selection import train_test_split

from mypipes_creditcard import *

warnings.filterwarnings('ignore')

In [2]:
df = pd.read_excel(r'/home/siddarth.jha@npci.org.in/Documents/Training/Data/default of credit card clients.xls', 
                  skiprows = [0])

In [3]:
df

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,29996,220000,1,3,1,39,0,0,0,0,...,88004,31237,15980,8500,20000,5003,3047,5000,1000,0
29996,29997,150000,1,3,2,43,-1,-1,-1,-1,...,8979,5190,0,1837,3526,8998,129,0,0,0
29997,29998,30000,1,2,2,37,4,3,2,-1,...,20878,20582,19357,0,0,22000,4200,2000,3100,1
29998,29999,80000,1,3,1,41,1,-1,0,0,...,52774,11855,48944,85900,3409,1178,1926,52964,1804,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   ID                          30000 non-null  int64
 1   LIMIT_BAL                   30000 non-null  int64
 2   SEX                         30000 non-null  int64
 3   EDUCATION                   30000 non-null  int64
 4   MARRIAGE                    30000 non-null  int64
 5   AGE                         30000 non-null  int64
 6   PAY_0                       30000 non-null  int64
 7   PAY_2                       30000 non-null  int64
 8   PAY_3                       30000 non-null  int64
 9   PAY_4                       30000 non-null  int64
 10  PAY_5                       30000 non-null  int64
 11  PAY_6                       30000 non-null  int64
 12  BILL_AMT1                   30000 non-null  int64
 13  BILL_AMT2                   30000 non-null  int64
 14  BILL_A

In [5]:
x = df.drop('default payment next month', axis = 1)
y = df[['default payment next month']]
df['MARRIAGE'].value_counts()

2    15964
1    13659
3      323
0       54
Name: MARRIAGE, dtype: int64

In [6]:
'''
SEX, MARRIAGE, : dummy variable
PAY_0-6: cyclic features

'''
colslist = x.columns
catvars = x[['SEX', 'MARRIAGE', 'EDUCATION']]
numvars = x[[x for x in colslist if x not in catvars]]
catvars.columns

Index(['SEX', 'MARRIAGE', 'EDUCATION'], dtype='object')

In [7]:
p1 = pdPipeline([
    ('var_select', VarSelector(['SEX', 'MARRIAGE', 'EDUCATION'])),
    ('create_dummy', dummy())
])

p2 = pdPipeline([
    ('var_select', VarSelector(numvars.columns)),
    ('missing_trt', MissingValues())
])

data_pipe = FeatureUnion([
    ('int_to_dummy', p1),
    ('impute_miss', p2)
])

In [8]:
X_train,X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [9]:
data_pipe.fit(X_train)


FeatureUnion(transformer_list=[('int_to_dummy',
                                pdPipeline(steps=[('var_select',
                                                   VarSelector(feature_names=['SEX',
                                                                              'MARRIAGE',
                                                                              'EDUCATION'])),
                                                  ('create_dummy', dummy())])),
                               ('impute_miss',
                                pdPipeline(steps=[('var_select',
                                                   VarSelector(feature_names=Index(['ID', 'LIMIT_BAL', 'AGE', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5',
       'PAY_6', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4',
       'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3',
       'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6'],
      dtype='object'))),
                                                  ('missing_trt',

In [10]:
len(data_pipe.get_feature_names())

31

In [11]:
data_pipe.get_feature_names()

['int_to_dummy__SEX_2',
 'int_to_dummy__MARRIAGE_2',
 'int_to_dummy__MARRIAGE_1',
 'int_to_dummy__MARRIAGE_3',
 'int_to_dummy__EDUCATION_2',
 'int_to_dummy__EDUCATION_1',
 'int_to_dummy__EDUCATION_3',
 'int_to_dummy__EDUCATION_5',
 'int_to_dummy__EDUCATION_4',
 'int_to_dummy__EDUCATION_6',
 'impute_miss__ID',
 'impute_miss__LIMIT_BAL',
 'impute_miss__AGE',
 'impute_miss__PAY_0',
 'impute_miss__PAY_2',
 'impute_miss__PAY_3',
 'impute_miss__PAY_4',
 'impute_miss__PAY_5',
 'impute_miss__PAY_6',
 'impute_miss__BILL_AMT1',
 'impute_miss__BILL_AMT2',
 'impute_miss__BILL_AMT3',
 'impute_miss__BILL_AMT4',
 'impute_miss__BILL_AMT5',
 'impute_miss__BILL_AMT6',
 'impute_miss__PAY_AMT1',
 'impute_miss__PAY_AMT2',
 'impute_miss__PAY_AMT3',
 'impute_miss__PAY_AMT4',
 'impute_miss__PAY_AMT5',
 'impute_miss__PAY_AMT6']

In [12]:
x_train = pd.DataFrame(data=data_pipe.transform(X_train),
                    columns=data_pipe.get_feature_names())
x_test=pd.DataFrame(data=data_pipe.transform(X_test),
                    columns=data_pipe.get_feature_names())

In [13]:
x_train

,int_to_dummy__SEX_2,int_to_dummy__MARRIAGE_2,int_to_dummy__MARRIAGE_1,int_to_dummy__MARRIAGE_3,int_to_dummy__EDUCATION_2,int_to_dummy__EDUCATION_1,int_to_dummy__EDUCATION_3,int_to_dummy__EDUCATION_5,int_to_dummy__EDUCATION_4,int_to_dummy__EDUCATION_6,...,impute_miss__BILL_AMT3,impute_miss__BILL_AMT4,impute_miss__BILL_AMT5,impute_miss__BILL_AMT6,impute_miss__PAY_AMT1,impute_miss__PAY_AMT2,impute_miss__PAY_AMT3,impute_miss__PAY_AMT4,impute_miss__PAY_AMT5,impute_miss__PAY_AMT6
0,1,1,0,0,1,0,0,0,0,0,...,78321,73731,39643,39457,3503,5001,2092,1218,1445,878
1,0,1,0,0,1,0,0,0,0,0,...,29155,25255,22001,0,5006,1244,851,955,0,0
2,1,0,1,0,0,0,0,1,0,0,...,850,0,6881,10340,0,850,0,6881,10340,182
3,0,1,0,0,0,1,0,0,0,0,...,38533,39639,39619,39140,2018,1900,2000,1500,1900,2000
4,1,1,0,0,1,0,0,0,0,0,...,114734,117823,120854,123904,4100,4200,5000,5000,5000,10700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23995,0,1,0,0,1,0,0,0,0,0,...,55739,55957,29238,6119,2000,3000,1591,72,1134,73421
23996,0,1,0,0,0,1,0,0,0,0,...,168386,164182,169029,172084,13500,6000,0,7500,6000,4000
23997,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23998,1,1,0,0,1,0,0,0,0,0,...,51152,35122,28633,28039,3000,2000,4500,1200,0,1200


In [14]:
x_test

,int_to_dummy__SEX_2,int_to_dummy__MARRIAGE_2,int_to_dummy__MARRIAGE_1,int_to_dummy__MARRIAGE_3,int_to_dummy__EDUCATION_2,int_to_dummy__EDUCATION_1,int_to_dummy__EDUCATION_3,int_to_dummy__EDUCATION_5,int_to_dummy__EDUCATION_4,int_to_dummy__EDUCATION_6,...,impute_miss__BILL_AMT3,impute_miss__BILL_AMT4,impute_miss__BILL_AMT5,impute_miss__BILL_AMT6,impute_miss__PAY_AMT1,impute_miss__PAY_AMT2,impute_miss__PAY_AMT3,impute_miss__PAY_AMT4,impute_miss__PAY_AMT5,impute_miss__PAY_AMT6
0,0,1,0,0,1,0,0,0,0,0,...,11581,12580,13716,14828,1500,2000,1500,1500,1500,2000
1,1,1,0,0,0,1,0,0,0,0,...,116684,101581,77741,77264,4486,4235,3161,2647,2669,2669
2,1,0,1,0,0,0,1,0,0,0,...,68530,69753,70111,70212,2431,3112,3000,2438,2500,2554
3,0,1,0,0,0,0,1,0,0,0,...,16172,16898,11236,6944,1610,1808,7014,27,7011,4408
4,1,1,0,0,1,0,0,0,0,0,...,42361,19574,20295,19439,2000,1500,1000,1800,0,1000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,1,1,0,0,0,1,0,0,0,0,...,830,0,1271,179,0,830,0,1271,179,1970
5996,1,1,0,0,0,1,0,0,0,0,...,111690,78070,78376,80912,6000,13151,0,3000,5000,10000
5997,1,0,1,0,0,1,0,0,0,0,...,48696,50385,52045,52661,2100,2500,2500,2500,1600,1500
5998,1,1,0,0,0,1,0,0,0,0,...,3967,1364,600,0,2494,3967,1370,600,0,0


In [15]:
print('Train-',x_train.shape)
print('Test-',x_test.shape)

Train- (24000, 31)
Test- (6000, 31)


### Logistic Regression

In [16]:
from sklearn.linear_model import LogisticRegression

In [26]:
params={'class_weight':['balanced',None],
        'penalty':['l1','l2'],
        'C':[.0001,.0005, .00001,.001,.005,.01,.05,.1,1]}

In [18]:
model=LogisticRegression(solver='liblinear')

In [19]:
from sklearn.model_selection import GridSearchCV

In [27]:
grid_search=GridSearchCV(model,
                         param_grid=params,
                         cv=10,
                         scoring="roc_auc",
                         n_jobs=-1,
                         verbose=20)

In [28]:
grid_search.fit(x_train,y_train)

Fitting 10 folds for each of 36 candidates, totalling 360 fits


GridSearchCV(cv=10, estimator=LogisticRegression(solver='liblinear'), n_jobs=-1,
             param_grid={'C': [0.0001, 0.0005, 1e-05, 0.001, 0.005, 0.01, 0.05,
                               0.1, 1],
                         'class_weight': ['balanced', None],
                         'penalty': ['l1', 'l2']},
             scoring='roc_auc', verbose=20)

In [29]:
grid_search.best_estimator_

LogisticRegression(C=1, class_weight='balanced', penalty='l1',
                   solver='liblinear')

In [23]:
logr=grid_search.best_estimator_

In [30]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.6f} (std: {1:.6f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [31]:
report(grid_search.cv_results_,5)

Model with rank: 1
Mean validation score: 0.723035 (std: 0.009485)
Parameters: {'C': 1, 'class_weight': 'balanced', 'penalty': 'l1'}

Model with rank: 2
Mean validation score: 0.722882 (std: 0.009694)
Parameters: {'C': 0.1, 'class_weight': 'balanced', 'penalty': 'l1'}

Model with rank: 3
Mean validation score: 0.722753 (std: 0.009899)
Parameters: {'C': 0.05, 'class_weight': 'balanced', 'penalty': 'l1'}

Model with rank: 4
Mean validation score: 0.722380 (std: 0.009773)
Parameters: {'C': 1, 'class_weight': None, 'penalty': 'l1'}

Model with rank: 5
Mean validation score: 0.722304 (std: 0.009857)
Parameters: {'C': 0.001, 'class_weight': 'balanced', 'penalty': 'l1'}



In [32]:
logr=LogisticRegression(fit_intercept=True,
                        **{'C': 1, 'class_weight': 'balanced', 'penalty': 'l1'},solver='liblinear')

In [33]:
logr.fit(x_train,y_train)

LogisticRegression(C=1, class_weight='balanced', penalty='l1',
                   solver='liblinear')

In [34]:
(logr.coef_[0]==0).sum()

1

In [35]:
list(zip(x_train.columns,logr.coef_[0]))

[('int_to_dummy__SEX_2', -0.12032929122209149),
 ('int_to_dummy__MARRIAGE_2', 0.33113252300777185),
 ('int_to_dummy__MARRIAGE_1', 0.4979383958702224),
 ('int_to_dummy__MARRIAGE_3', 0.3760490319057653),
 ('int_to_dummy__EDUCATION_2', 0.1340523252509528),
 ('int_to_dummy__EDUCATION_1', 0.18422338319276493),
 ('int_to_dummy__EDUCATION_3', 0.10784294541962201),
 ('int_to_dummy__EDUCATION_5', -1.0863383927899788),
 ('int_to_dummy__EDUCATION_4', -0.5715930485536523),
 ('int_to_dummy__EDUCATION_6', 0.0),
 ('impute_miss__ID', -1.713729861040158e-06),
 ('impute_miss__LIMIT_BAL', -7.637576188171921e-07),
 ('impute_miss__AGE', 0.007240366033879539),
 ('impute_miss__PAY_0', 0.5068052629606166),
 ('impute_miss__PAY_2', 0.0986324255576443),
 ('impute_miss__PAY_3', 0.05671976027650273),
 ('impute_miss__PAY_4', 0.04192518874956066),
 ('impute_miss__PAY_5', 0.007623533238434646),
 ('impute_miss__PAY_6', -0.006326062402243039),
 ('impute_miss__BILL_AMT1', -4.54124111027759e-06),
 ('impute_miss__BILL_AMT

In [36]:
logr.predict_proba(x_test)

array([[0.48133687, 0.51866313],
       [0.60395288, 0.39604712],
       [0.5114592 , 0.4885408 ],
       ...,
       [0.48650836, 0.51349164],
       [0.42399407, 0.57600593],
       [0.62799124, 0.37200876]])

In [37]:
logr.classes_

array([0, 1])

In [38]:
cutoffs=np.linspace(0.01,0.99,99)

cutoffs

array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33,
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65, 0.66,
       0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77,
       0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88,
       0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99])

In [39]:
logr.predict_proba(x_train)

array([[0.56965518, 0.43034482],
       [0.48826098, 0.51173902],
       [0.78404214, 0.21595786],
       ...,
       [0.77545592, 0.22454408],
       [0.56260478, 0.43739522],
       [0.79413951, 0.20586049]])

In [40]:
logr.classes_

array([0, 1])

In [41]:
train_score=logr.predict_proba(x_train)[:,1]
real=y_train
 # In  order to find the probability of which column is for outcome 1 and which for outcome 0


In [42]:
(train_score>0.2).astype(int)

array([1, 1, 1, ..., 1, 1, 1])

In [43]:
KS_all=[]

for cutoff in cutoffs:
    
    predicted=(train_score>cutoff).astype(int)

    TP=((predicted==1) & (real==1)).sum()
    TN=((predicted==0) & (real==0)).sum()
    FP=((predicted==1) & (real==0)).sum()
    FN=((predicted==0) & (real==1)).sum()
    
    P=TP+FN
    N=TN+FP
    
      
    KS=(TP/P)-(FP/N)
       
    KS_all.append(KS)
    

ValueError: Unable to coerce to Series, length must be 1: given 24000